# Import Libraries

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import cv2
import pandas as pd
import numpy as np
import ntpath
import io
import tqdm
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import layers, optimizers, models
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from imgaug import augmenters

from IPython.display import Markdown

def bold(string):
    display(Markdown("**" + string + "**"))

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Load Data

In [ ]:
root_dir = '/mnt/d/Datasets/track-master/'
driving_log_file_path = '/mnt/d/Datasets/track-master/driving_log.csv'
driving_images_file_path = '/mnt/d/Datasets/track-master/IMG/'

In [ ]:
names = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']

In [ ]:
df = pd.read_csv(driving_log_file_path, names=names)
df.head()

In [ ]:
def df_stats(data):
    bold(" SHAPE ".center(50, "#"))
    print("ROWS: {}".format(data.shape[0]))
    print("COLS: {}".format(data.shape[1]))
    bold(" TYPES ".center(50, "#"))
    print(data.dtypes)
    bold(" MISSING VALUES ".center(50, "#"))
    print(data.isnull().sum())
    bold(" DUPLICATED VALUES ".center(50, "#"))
    print("NUMBER OF DUPLICATED VALUES: {}".format(data.duplicated().sum()))
    #bold(" DESCRIBE ".center(50, "#"))
    #print(data.describe().T)
    bold(" MEMORY USAGE ".center(50, "#"))
    buf = io.StringIO()
    data.info(buf=buf)
    info = buf.getvalue().split("\n")[-2].split(":")[1].strip()
    print("Memory Usage: {}".format(info))

In [ ]:
df_stats(df)

# EDA

In [ ]:
num_bins = 25
samples_per_bin = 400
hist, bins = np.histogram(df["steering"], num_bins)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.05)
plt.plot(
    (np.min(df["steering"]), np.max(df["steering"])),
    (samples_per_bin, samples_per_bin),
)

# Preprocess

In [ ]:
remove_list = []
for j in range(num_bins):
    list_ = np.where((df["steering"] >= bins[j]) & (df["steering"] <= bins[j + 1]))[0]
    list_ = shuffle(list_)[samples_per_bin:]
    remove_list.extend(list_)

In [ ]:
print('Before Removing:', len(df))

In [ ]:
df.drop(df.index[remove_list], inplace=True)

In [ ]:
print('After Removing:', len(df))

In [ ]:
hist, bins = np.histogram(df["steering"], num_bins)
center = (bins[:-1] + bins[1:]) * 0.5
plt.bar(center, hist, width=0.05)
plt.plot(
    (np.min(df["steering"]), np.max(df["steering"])),
    (samples_per_bin, samples_per_bin),
)

In [ ]:
image_paths = []
steerings = []

for idx, row in tqdm.tqdm(df.iterrows()):
    center = row['center']
    image_paths.append(center)
    steerings.append(float(row['steering']))
    
    left = row['left']
    image_paths.append(left)
    steerings.append(float(row['steering']) + 0.15)
    
    right = row['right']
    image_paths.append(right)
    steerings.append(float(row['steering']) - 0.15)

In [ ]:
image_paths = np.array(image_paths)
steerings = np.array(steerings)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(image_paths, steerings, test_size=0.1, random_state=42)

In [ ]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
def random_augment(image, steering_angle):
    image = mpimg.imread(image)
    if np.random.rand() < 0.5:
        pan = augmenters.Affine(translate_percent={'x': (-0.1, 0.1), 'y': (-0.1, 0.1)})
        image = pan.augment_image(image)

    if np.random.rand() < 0.5:
        zoom = augmenters.Affine(scale=(1, 1.3))
        image = zoom.augment_image(image)

    if np.random.rand() < 0.5:
        brightness = augmenters.Multiply((0.2, 1.2))
        image = brightness.augment_image(image)

    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
        steering_angle = -steering_angle

    return image, steering_angle

In [ ]:
def preprocess_img(img):
    img = img[60:135, :, :]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img, (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    return img / 255.0

In [ ]:
def batch_generator(image_paths, steering_angles, batch_size, is_training):
    while True:
        batch_images = []
        batch_steerings = []

        for i in range(batch_size):
            random_index = random.randint(0, len(image_paths) - 1)
            image_idx = image_paths[random_index]
            image = mpimg.imread(image_idx)
            steering = steering_angles[random_index]

            if is_training:
                image, steering = random_augment(image_idx, steering)

            image = preprocess_img(image)
            batch_images.append(image)
            batch_steerings.append(steering)

        yield np.asarray(batch_images), np.asarray(batch_steerings)

In [ ]:
train_generator = batch_generator(X_train, y_train, 100, 1)
test_generator = batch_generator(X_test, y_test, 100, 0)

# Model

In [ ]:
model = models.Sequential([
    layers.Lambda(lambda x: x / 255.0, input_shape=(66, 200, 3)),

    layers.Conv2D(24, kernel_size=(5, 5), strides=(2, 2), activation='relu'),
    layers.Conv2D(36, kernel_size=(5, 5), strides=(2, 2), activation='relu'),
    layers.Conv2D(48, kernel_size=(5, 5), strides=(2, 2), activation='relu'),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),

    layers.Dropout(0.5),

    layers.Flatten(),

    layers.Dense(100, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(50, activation='relu'),
    layers.Dense(10, activation='relu'),

    layers.Dense(1)
])

In [ ]:
model.compile(loss='mse', optimizer=optimizers.Adam(learning_rate=1e-3))

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_layer_names=True, show_shapes=True)

# Train

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 300,
    validation_data = test_generator,
    validation_steps = 200,
    epochs = 25
)

# Results

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.head()

In [ ]:
plt.figure()
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(["train", "valid"])
plt.title("Loss Curve")
plt.show()

In [ ]:
model.save('self_driving_car_model.h5')